In [23]:
import pandas as pd
from deta import Deta
import time
import random
from datetime import datetime, timedelta

In [24]:
DETA_KEY = 'e0jeqtjtszl_L996atb2Q4i8HKS36bU6drgU6tt1HYSq' 

# Initialize with a project key
deta = Deta(DETA_KEY)


# This is how to create/connect a database
db = deta.Base("data_co2")
# db = deta.Base("users")

In [11]:
def fetch_all_co2_data():
    db = deta.Base("data_co2")
    res = db.fetch()
    all_items = res.items

    while res.last:
        res = db.fetch(last=res.last)
        all_items += res.items
    return all_items

data_co2 = pd.DataFrame(fetch_all_co2_data())
data_co2

,aula,datetime,edificio,key,medicion,piso,sensor
0,aula 1,02-06-2022 00:00:00,capruis y favuis,0,972,1,aula 1_sensor
1,aula 1,03-06-2022 00:00:00,camilo torres,1,805,1,aula 1_sensor
2,aula 1,12-06-2022 00:00:00,capruis y favuis,10,405,1,aula 1_sensor
3,aula 1,10-09-2022 00:00:00,daniel casas,100,466,1,aula 1_sensor
4,aula 4,13-09-2022 00:00:00,cenivam,1000,531,2,aula 4_sensor
...,...,...,...,...,...,...,...
2686,aula 4,08-09-2022 00:00:00,cenivam,995,687,2,aula 4_sensor
2687,aula 4,09-09-2022 00:00:00,daniel casas,996,1003,2,aula 4_sensor
2688,aula 4,10-09-2022 00:00:00,capruis y favuis,997,746,2,aula 4_sensor
2689,aula 4,11-09-2022 00:00:00,camilo torres,998,1079,2,aula 4_sensor


In [12]:
data_co2['datetime'] = pd.to_datetime(data_co2['datetime'])

In [ ]:
data = fetch_all_co2_data()
df_data_co2 = pd.DataFrame(data_co2)
df_data_co2

In [19]:
def create_dummy_date():

    aulas = ['aula 1','aula 2','aula 3','aula 4','aula 5','aula 6','aula 7','aula 8','aula 9']
    edificios = ['CAMILO TORRES', 'CAPRUIS Y FAVUIS','CENTIC','DANIEL CASAS','CENIVAM']


    dict_dummy = []
    start_date = datetime.strptime('01/06/2022', '%d/%m/%Y')
    for i in aulas:
        piso = 0
        n_aula = int(i.split(' ')[1] ) 
        if n_aula <=3:
            piso = 1
        elif n_aula >3 and n_aula <=6:
            piso = 2
        elif n_aula > 6 :
            piso = 3
        
        for dia in range(1,300):
            edificio = random.choice(edificios)
            dict_ = {
                'aula':i,
                'medicion': random.randint(400,1200),
                'datetime': start_date + timedelta(dia),
                'sensor' : f'{i}_sensor',
                'edificio' : edificio,
                'piso' : piso,
            }
            dict_dummy.append(dict_)
        df = pd.DataFrame(dict_dummy)
    return df

In [21]:
df = create_dummy_date()
df.datetime.max()
len(df)

2691

In [25]:
contador = 0
for i in df.to_dict(orient = 'records'):
    dict_temp = {
        'key' : str(contador),
        'medicion' : int(i['medicion']),
        'aula' : str(i['aula']),
        'datetime' :i['datetime'].strftime('%d-%m-%Y'),
        'sensor' : i['sensor'],
        'edificio' :i['edificio'].lower(),
        'piso' : int(i['piso']),
    }
    db.put(dict_temp)
    print(contador, end = '\r')
    contador +=1

In [ ]:
lista = [{
    "user":"ivan",
    "password":"ivan123"
},
{
    "user":"javier",
    "password":"javier123"
}
]

df = pd.DataFrame(lista)

In [ ]:
with open('auth.json', 'w') as f:
    f.write(df.to_json( orient='records', lines = True))


In [ ]:
from datetime import datetime
user_nickname = 'carlos'
user_name = 'carlos jose'
user_last_name = 'maestre muza'
user_password = 'carlos123'
user_date_registration = datetime.now().strftime('%d/%m/%Y')
user_role = 'Administrator'

def new_user(user,password,name,last_name,role,date_login):
    new_user_data = {
        'user' : user,
        'password' : password,
        'name' : name,
        'last_name' : last_name,
        'role' : role,
        'date_login' : date_login,
    }

    df_temp = pd.DataFrame([new_user_data])
    df = pd.read_json('auth.json', lines=True)
    df = pd.concat([df,df_temp])

    with open('auth.json', 'w') as f:
        f.write(df.to_json( orient='records', lines = True))

In [ ]:
from db import create_new_user
x = [{'user': 'ivan',
  'password': 'ivan123',
  'name': 'Ivan Daniel',
  'last_name': 'Maestre Muza',
  'role': 'administrator'},
 {'user': 'javier',
  'password': 'javier123',
  'name': 'Javier Javier',
  'last_name': 'Daza Olivella',
  'role': 'administrator'},
 {'user': 'eladio',
  'password': 'eladio123',
  'name': 'Eladio',
  'last_name': 'Carrión Morales',
  'role': 'viewer'}]

for i in x:
    user_name = i['user']
    password = i['password']
    name = i['name']
    last_name = i['last_name']
    role = i['role']
    create_new_user(user_name,password,name,last_name,role)

In [ ]:


def authentication(user:str, password:str):
    response = False
    df = pd.read_json('auth.json', lines=True)
    df_dict = df.to_dict(orient='records')

    for dictionary in df_dict:

        if dictionary['user'] == user and dictionary['password'] == password:
            response = True

    return response
            
def fetch_all_users():
    """Returns a dict of all users"""
    res = db.fetch()
    return res.items


In [ ]:
user = 'javier'
password = 'javier123'

authentication(user=user,password=password)

In [ ]:
df = pd.read_json('auth.json', lines=True)
df_dict = df.to_dict(orient='records')
df_dict

In [ ]:
from deta import Deta
from datetime import datetime
DETA_KEY = 'e0jeqtjtszl_L996atb2Q4i8HKS36bU6drgU6tt1HYSq' 

# Initialize with a project key

deta = Deta(DETA_KEY)


# This is how to create/connect a database
db = deta.Base("users")

def create_new_user(user_name,password,name,last_name,role):
    # create new user
    date_login = datetime.now().strftime('%d-%m-%Y')
    return db.put({"key":user_name, "password":password, "name":name.title().strip(), "last_name": last_name.title().strip(),"role": role.title().strip(),"date_login":date_login})


def update_user(user_name,password,name,last_name,role):
    return db.update({"password":password, "name":name.title().strip(), "last_name": last_name.title().strip(),"role": role.title().strip()}, key = user_name)

def fetch_all_users():
    """Returns a dict of all users"""
    res = db.fetch()
    return res.items


def get_user(user_name:str):
    return db.get(user_name)


def authentication(user_name_input:str, password_input):
    user_fetched = get_user(user_name_input)
    if user_fetched != None:
        password_fetched = user_fetched['password']
        if password_input == password_fetched:
            password = user_fetched['password']
            name = user_fetched['name']
            last_name = user_fetched['last_name']
            role = user_fetched['role']
            auth_flag = True
            return auth_flag,password,name,last_name,role
        else:
            password = ''
            name = ''
            last_name = ''
            role = ''
            auth_flag = False
            return auth_flag,password,name,last_name,role




In [ ]:
key = "ivan"
password = "ivan123"
name = "Ivan daniel"
last_name = "maestre muza"
role = "administrator"


# if get_user(key) != None:
#     auth_flag,password,name,last_name,role = authentication(user_name_input= key, password_input = password)
# else:
#     print('The user dont exists')
create_new_user(key,password,name,last_name,role)

In [ ]:
def change_password(user_name_input,password_input,new_password_input:str):
    user_fetched = get_user(user_name_input)
    if user_fetched != None:
        try:
            db.update({"password":password_input},key=user_name_input)
            return True
        except Exception as e:
            print(e)
            return False

In [ ]:
user_name_input = 'efra'
password_input = 'efra123'
new_password_input = 'efra124'


user_fetched = get_user(user_name_input)
if user_fetched != None:
    user_dict = db.get(user_name_input)
    if user_dict['password'] == password_input:
        try:
            db.update({"password":new_password_input},key=user_name_input)
            retorno =  True
        except Exception as e:
            print(e)
            retorno =  False
else:
    retorno = False

In [ ]:
retorno

In [ ]:
x = db.update({"password":'efra123'},key='efra')

In [ ]:
db.get('efra')

In [ ]:
from db import change_password
change_password('efra','efra124','eee')

In [ ]:
x = pd.DataFrame(fetch_all_users())
x = x[x['key'] == 'ivan']
y = x.to_dict(orient='records')
y

In [ ]:
import random
from datetime import datetime, timedelta
aulas = ['aula 1','aula 2','aula 3','aula 4','aula 5']

dict_dummy = []
start_date = datetime.strptime('01/01/2023', '%d/%m/%Y')
for i in aulas:
    for dia in range(1,50):
        dict_ = {
            'aula':i,
            'medicion': random.randint(500,1000),
            'datetime': start_date + timedelta(dia),
            'sensor' : f'{i}_sensor'
        }
        dict_dummy.append(dict_)

start_date = datetime.strptime('01/01/2023', '%d/%m/%Y')
end_date = datetime.strptime('20/02/2023', '%d/%m/%Y')
end_date - start_date

In [ ]:
df = pd.DataFrame(dict_dummy)

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.02)
aula = 'aula 1'
df = df[df['aula'] == f'{aula}']
fig = px.line(df, x="datetime", y="medicion",  color='aula')

fig.update_layout(title_text=f'Line Plot CO2 Levels \n Aula: {aula}', title_x=0.5,xaxis_title =  'Date',yaxis_title = 'Nivel de CO2',legend_title = 'Aula')
fig.add_hline(y=800,line=dict(color="#EB4747"))

fig.show()


In [ ]:
df_temp = df[df['aula'] == 'aula 4']
df_temp['medicion'].mean()

In [ ]:
df_temp = df[df['aula'] == 'aula 1']
df_temp.sort_values(by = ['datetime'], ascending = False, inplace =True)
df_temp.head()

In [ ]:
fig = px.histogram(df_temp, x="medicion", nbins=20)
fig.update_layout(title_text=f'Histogram Plot CO2 Levels \n Aula: {aula}', title_x=0.5,xaxis_title =  'Date',yaxis_title = 'Nivel de CO2',legend_title = 'Aula')
fig.show()

In [ ]:
fig = px.bar(df_temp, x='datetime', y='medicion')
fig.update_traces(marker_color='green')
fig.show()

In [ ]:
pd.DataFrame(fetch_all_users())

In [ ]:
aulas = ['aula 1','aula 2','aula 3','aula 4','aula 5','aula 6','aula 7','aula 8','aula 9']
edificios = ['CAMILO TORRES', 'CAPRUIS Y FAVUIS','CENTIC','DANIEL CASAS','CENIVAM']
from datetime import datetime, timedelta
import random
dict_dummy = []
start_date = datetime.strptime('31/12/2022', '%d/%m/%Y')
for i in aulas:
    piso = 0
    n_aula = int(i.split(' ')[1] ) 
    if n_aula <=3:
        piso = 1
    elif n_aula >3 and n_aula <=6:
        piso = 2
    elif n_aula > 6 :
        piso = 3
    
    for dia in range(1,51):
        edificio = random.choice(edificios)
        dict_ = {
            'aula':i,
            'medicion': random.randint(400,1000),
            'datetime': start_date + timedelta(dia),
            'sensor' : f'{i}_sensor',
            'edificio' : edificio,
            'piso' : piso,
        }
        dict_dummy.append(dict_)
    df = pd.DataFrame(dict_dummy)

In [ ]:
df.groupby(['edificio'], as_index = False)['aula'].max()

In [ ]:
x = df[(df['edificio'] == 'CENTIC') & (df['piso'] == 1)]
df_temp_dict = x.to_dict(orient = 'records')
for i in df_temp_dict:
    print(i['aula'])

In [ ]:
import requests

In [ ]:
r = requests.get('"localhost:8091/device/?uuid=1')

In [ ]:
from pymongo import MongoClient
import requests

payload = {
    'deviceUUID' : '1',
    'topic' : 'piso',
    'timeStamp' : '25-03-2023 12:42:57',
    'data' : '5',
}
# payload = {
#     'deviceUUID' : '1',
#     'aula': 'aula 1',
#     'medicion': 756,
#     'datetime': '22-03-2023 12:42:57',
#     'sensor': 'aula 1_sensor',
#     'edificio': 'CENTIC',
#     'piso': 1}


connect_url = "mongodb://192.34.56.224:27017/" 
client = MongoClient(connect_url)
db = client.get_database('iot')
collection = db.get_collection('messages')
x = collection.insert_one(payload)

In [ ]:

## DELETE DOCUMENT 
myquery = { "deviceUUID": "5" } # delete_one according to de deviceUUID (delete_one)
myquery = { "deviceUUID": {"$regex": "^1"} } # delete all the documents that deviceUUID starst with 1 (delete_many)

collection.delete_many(myquery)

In [ ]:
# insert bulk
collection.insert_many(df.to_dict('records'))

In [ ]:
import pandas as pd

r = requests.get('http://192.34.56.224:8091/device/?uuid=1')
df = pd.DataFrame(r.json())

In [ ]:
final_dict = {}
for i in r.json():
    dict_temp = {
        'deviceUUID' : i['deviceUUID'],
       'data' : {}
    }
    final_dict.append(dict_temp)

In [ ]:
x = [{
    'deviceUUID' : '1',
    'data' : {'temp':'1', 'co2': '2'}
},{
    'deviceUUID' : '2',
    'data' : {'temp':'6', 'co2': '20'}
}]

y = pd.DataFrame(x)
y

In [ ]:
def create_dummy_date():
    from datetime import datetime,timedelta
    import random
    aulas = ['aula 1','aula 2','aula 3','aula 4','aula 5','aula 6','aula 7','aula 8','aula 9']
    edificios = ['CAMILO TORRES', 'CAPRUIS Y FAVUIS','CENTIC','DANIEL CASAS','CENIVAM']


    dict_dummy = []
    start_date = datetime.strptime('30/06/2022', '%d/%m/%Y')
    for i in aulas:
        piso = 0
        n_aula = int(i.split(' ')[1] ) 
        if n_aula <=3:
            piso = 1
        elif n_aula >3 and n_aula <=6:
            piso = 2
        elif n_aula > 6 :
            piso = 3
        
        for dia in range(1,300):
            edificio = random.choice(edificios)
            dict_ = {
                'aula':i,
                'nivel_co2': random.randint(400,1200),
                'datetime': (start_date + timedelta(dia)).strftime('%d/%m/%Y %H:%M:%S'),
                'sensor' : f'{i}_sensor',
                'edificio' : edificio,
                'piso' : piso,
            }
            dict_dummy.append(dict_)
        df = pd.DataFrame(dict_dummy)
    return df

df = create_dummy_date()

In [ ]:
df

In [ ]:
dict_final = []
df_dict = df.to_dict(orient='records')

for i in df_dict:

    deviceUUID = f"{i['edificio'].lower()}_piso_{i['piso']}_{i['aula']}"
    keys = i.keys()
    for j in keys:
        dict_temp = {
            'deviceUUID' : deviceUUID,
            'topic' : j,
            'data' : i[j]
        }
        dict_final.append(dict_temp)

In [ ]:
dict_final[0:10]

In [ ]:
x = pd.DataFrame(dict_final)

In [ ]:
Problemas con el schema en la base de datos.
estaremos recibiendo mediciones de los diferentes edificios, pisos y aulas, personalmente no recomiendo el uso de base de datos NoSQL para la herramienta.
sin embargo, si queremos mantener el mismo tipo de base de datos no relacional, sugiero el siguiente schema:
{
    'deviceUUID' : '1', # indicador unico para cada sensor
    'aula': 'aula 1', # Aula de la medicion
    'nivel_co2': 756, # valor de la medicion de CO2
    'datetime': '22-03-2023 12:42:57', # %d-%M-%Y %H:%M:%S datetime del momento de la medicion
    'edificio': 'CENTIC', # edificio de la medicion
    'piso': 1 # piso de la medicion
}
por que? se pretende tener varias mediciones para diferentes dias, la llave no podra ser siempre el deviceUUID, ya que tendremos muchas mediciones para ese deviceUUID
lo que genera un desorden al tener que agregar 1 documento para cada atributo de nuestra medicion, ya que solo tenemos 1 key "topico" que es el que almacenara la informacion de cada medicion, no solo el nivel de co2, sino tmb informacion relacionada a la ubicacion del sensor

el API REST debera recibir mas parametros para la consulta, no solo el deviceUUID, podria recibir criterios opcionales tales como el aula, el edificio y el piso.
los criterios como el edificio, piso y aula se pueden obtener desde el deviceUUID, pero eso nos condiciona a tener un diccionario de cada deviceUUID con su informacion
de la ubicacion de cada salon, y asu vez, obtener edificio, piso y # de aula.


In [ ]:
payload = {
    'deviceUUID' : '1',
    'topic' : 'nivel_co2',
    'timeStamp' : '22-03-2023 12:42:57',
    'data' : '14',
}
payload = {
    'deviceUUID' : '1',
    'aula': 'aula 1',
    'medicion': 756,
    'datetime': '22-03-2023 12:42:57',
    'sensor': 'aula 1_sensor',
    'edificio': 'CENTIC',
    'piso': 1}